# The Lattice Boltzmann model 
By: Tom de Krom (4281594) and Vincent Heusinkveld (4390725)

---
## Introduction 
In this project the Lattice Boltzmann model is considered. It is a so called 'toy model', solely based on the laws of energy, mass and momentum conservation. And it uses the isotropic relaxation of stress.  

---
## The Lattice Boltzmann model 
In the model only densities of particles, $n_i$, and velocity, $v_i$, with a certain direction are taken into account. These directions correspond to the vectors labeled i making up the lattice. In this case a 9 vector lattice is used called the d2q9 lattice as shown in the figure below (Thijssen 2007): 

![d2q9_lattice](./images/d2q9.png)

### The algorithm

The algorithm works as follows (Thijssen 2007):

- The densities $n_i$ are moved to the appropriate neighbour according to the direction of the lattice vectors.
- If the velocities pass over the boundaries they should be reversed.
- The velocities are calulated at each point and are weighted according to their respective density.
- A small velocity is added in the flow direction to simulate a pressure gradient. 
- The equilibrium distribution is calulated according to formula 14.34 of Thijssen 2007.
- The densites are relaxed at each point according to: $n_i^{new} = (1-1/\tau)n_i^{old} + n^{eq}_i/\tau$. In which $\tau$ is the relaxation time.

### Usefull relations 

The relation between the viscocity and the relaxation time is given by: $\nu = \frac{2\tau - 1}{6}\frac{\Delta x^2}{\Delta t}$. (Thijssen 2007)

The curvature of the velocity of the flow should be $\nabla P/\rho v$. In which $\nabla P$ can be related to the velocity addition in the flow direction via: $ \nabla P = c\rho\Delta v $. (Thijssen 2007)



---
## The simulation 
### The setup
For the geometry a 2D 'pipe' is defined, to which a square obstruction can be added, which will be centered in the pipe. Then the lattice vectors are defined, including a relation between the lattice vector and it's inverse (version in opposite direction). This is later need to incorporate the boundary conditions. 

As for the parameters, $\Delta x$ and $\Delta t$ are set to 1 to simplify the relations. The maximum number of simulation iterations and the viscosity can be set to the desired value. The viscosity should be defined such that the relaxation time, $\tau$, is bigger then 1. This is to ensure that the system does not relax faster then the minimal time step, $\Delta t$. 

### Application of the algorithm
The simulation is initialised such that the density everywhere equals 1 expect for the boundaries and in the obstruction, at which it is set to 0. Then a velocity forcing in applied (due to the pressure gradient), after which the equilibrium distribution, $n_{eq}$, is determined for every lattice point.

After this, the algorithm described above is implemented. For the translation of the densities, `numpy.roll()` is used in combination with the lattice vectors. 

The boundary condition has the following worling principle:
If the density corresponding to vector `[1,0]` crosses the right boundary, the density gets transferred to the `[-1,0]` vector, such that the next time step the density returns to the pipe domain. 

The rest of the code is implemented straightforwardly. 


---
## Results 
- Show parabolic behaviour
- Someting with Reynolds number?
- Quantatative restuls including movies 

---
## Conclusion 

---
## Code checks

### Parabolic flow profile
### Conservation
**Energy:**

**Density:**

---
## Simulation performance
Scaling with system size. Absolute runtime. 


---
## Collaboration 
In general...

**Both:**

**Tom:**

**Vincent:**


---
## Sources
- Thijssen, J. (2007). Computational physics, 2nd edition. Camebridge University Press